<a href="https://colab.research.google.com/github/alexlopespereira/machine_learning/blob/main/Notebooks/Aula2/Aula2_01_CarsPricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação de Preço de Automóveis em Faixas
### Usando Rede Neural (MLPClassifier) do scikit-learn

Este *notebook* realiza a classificação do preço de automóveis em três faixas (baixo, médio, alto), definidas pelos percentis de 33% e 66%. São realizadas as seguintes etapas:

1. **Carregamento** e **exploração inicial** dos dados.
2. **Limpeza** de dados (tratamento de valores faltantes, conversão de tipos).
3. **Categorização** da variável *price* em faixas.
4. **Análise exploratória** simples (histograma e mapa de calor de correlação).
5. **Conversão** de variáveis categóricas em *dummies* (One-Hot Encoding).
6. **Normalização** das variáveis numéricas (MinMaxScaler).
7. **Seleção de variáveis** (RFE) usando *LogisticRegression* como estimador base.
8. **Rede Neural MLP** para classificação (MLPClassifier do scikit-learn).
9. **Avaliação** do modelo (matriz de confusão, relatório de classificação, acurácia).

> **Observação**: O RFE utiliza `LogisticRegression` internamente, pois o `MLPClassifier` não disponibiliza coeficientes/atributos de importância de forma direta para a seleção de variáveis.

Vamos iniciar o processo!

## 1. Importação das Bibliotecas
*Nesta etapa, importamos as bibliotecas Python necessárias para o projeto.*

**Detalhes sobre algumas funções**:
- `warnings.filterwarnings('ignore')`: suprime avisos que podem poluir a saída.
- `pandas` (`pd`): manipulação de dataframes.
- `numpy` (`np`): operações matemáticas/vetoriais.
- `matplotlib.pyplot` (`plt`) e `seaborn` (`sns`): geração de gráficos e visualizações.
- `LogisticRegression`: modelo linear usado aqui somente para o processo de RFE.
- `MLPClassifier`: rede neural *Multilayer Perceptron* para classificação.
- `RFE`: *Recursive Feature Elimination*, método de seleção de variáveis.
- `MinMaxScaler`: normalização dos dados para o intervalo [0,1].
- `train_test_split`: separa dados em conjunto de treino e teste.
- `classification_report`, `confusion_matrix`, `accuracy_score`: métricas de avaliação.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

## 2. Carregamento e Breve Exploração do Dataset
*Nesta seção, carregamos o arquivo CSV com os dados dos automóveis e fazemos uma exploração inicial.*

**Funções e argumentos**:
- `pd.read_csv('Automobile.csv')`: lê o arquivo CSV e retorna um dataframe.
- `df.head(10)`: exibe as primeiras 10 linhas do dataframe.
- `df.info()`: mostra resumo das colunas, tipos e quantidade de dados não-nulos.
- `df.describe()`: gera estatísticas descritivas para colunas numéricas.

In [ ]:
# Carregar o arquivo CSV (ajuste o caminho se necessário)
df = pd.read_csv('Automobile.csv')

print("Amostra do dataset original:")
display(df.head(10))

print("\nInformações do dataset:")
print(df.info())

print("\nDescrição estatística das colunas numéricas:")
display(df.describe())

## 3. Limpeza de Dados
*Substituímos valores faltantes (`?`) por `NaN`, removemos linhas com dados ausentes e convertemos colunas numéricas.*

**Funções e argumentos**:
- `df.replace('?', np.nan, inplace=True)`: substitui `?` por `NaN` no próprio dataframe.
- `df.dropna(axis=0, how='any', inplace=True)`: remove linhas com qualquer valor faltante.
- `pd.to_numeric(..., errors='coerce')`: converte para número, transformando valores inválidos em `NaN`.

In [ ]:
# Substituir valores '?' por NaN
df.replace('?', np.nan, inplace=True)

# Definir colunas que devem ser numéricas
numerical_cols = ['price', 'bore', 'stroke', 'horsepower', 'peak-rpm']
for col in numerical_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Remover registros com valores nulos
df.dropna(axis=0, how='any', inplace=True)

print("\nTamanho do dataframe após remoção de NaNs:", df.shape)
print("\nDescrição estatística após limpeza:")
display(df.describe())

print("\nVerificar tipos de dados pós-limpeza:")
print(df.dtypes)

## 4. Criação da Variável Alvo (Categorização do Preço)
*Definimos duas faixas de corte (percentis de 33% e 66%) para classificar o preço em: 'low', 'medium' ou 'high'.*

**Funções e argumentos**:
- `np.percentile(df['price'], 33)`: retorna o valor do 33º percentil da coluna `price`.
- `df.apply(...)`: aplica uma função em cada valor da coluna `price`.

In [ ]:
# Calcular os percentis 33 e 66
p33 = np.percentile(df['price'], 33)
p66 = np.percentile(df['price'], 66)

print(f"\nValor do 33o percentil: {p33:.2f}")
print(f"Valor do 66o percentil: {p66:.2f}")

def categorize_price(x):
    if x <= p33:
        return 'low'
    elif x <= p66:
        return 'medium'
    else:
        return 'high'

df['price_category'] = df['price'].apply(categorize_price)

print("\nDistribuição das categorias de preço:")
display(df['price_category'].value_counts())

## 5. Análise Exploratória Básica
*Exibimos um histograma para visualizar a distribuição de preços e um mapa de calor das correlações entre variáveis numéricas.*

**Funções e argumentos**:
- `sns.histplot(...)`: gera um histograma com opção de curva *kde*.
- `sns.heatmap(...)`: mostra uma matriz de correlação com coloração, `annot=True` exibe valores numéricos.
- `corr()`: calcula a correlação de Pearson entre as colunas numéricas de um DataFrame.

In [ ]:
# Histograma do preço
plt.figure(figsize=(8, 6))
sns.histplot(df['price'], kde=True, color='green')
plt.title('Distribuição do Preço (Histograma)')
plt.xlabel('Preço')
plt.ylabel('Frequência')
plt.show()

# Mapa de calor de correlação
numerical_cols_2 = ['wheel-base','length','width','height','curb-weight',
                    'engine-size','bore','stroke','compression-ratio','horsepower',
                    'peak-rpm','city-mpg','highway-mpg','price']
corr_matrix = df[numerical_cols_2].corr()

plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, annot=True, cmap='YlGnBu', fmt=".2f")
plt.title('Mapa de Calor de Correlação')
plt.show()

## 6. Conversão de Variáveis Categóricas em *Dummies*
*Usamos `pd.get_dummies` para criar variáveis binárias (0/1) a partir de colunas categóricas.*

**Funções e argumentos**:
- `pd.get_dummies(df[cat_cols], drop_first=True)`: cria colunas *dummy* para cada categoria, removendo a primeira (referência) para evitar a *dummy trap*.
- `pd.concat([...], axis=1)`: concatena dataframes lado a lado.

In [ ]:
# Definir colunas categóricas
cat_cols = ['make','fuel-type','aspiration','num-of-doors','body-style',
            'drive-wheels','engine-location','engine-type','num-of-cylinders',
            'fuel-system']

# Criar dummies
df_dummies = pd.get_dummies(df[cat_cols], drop_first=True)
df = pd.concat([df, df_dummies], axis=1)

# Remover as colunas categóricas originais
df.drop(columns=cat_cols, inplace=True)

print("\nExemplo das colunas dummy:")
display(df_dummies.head(5))

## 7. Normalização das Variáveis Numéricas
*Aplicamos `MinMaxScaler` para escalar cada coluna numérica para o intervalo [0,1].*

**Funções e argumentos**:
- `MinMaxScaler(feature_range=(0,1))`: (padrão) transforma cada valor x em (x - min) / (max - min).
- `scaler.fit_transform(...)`: primeiro *aprende* (fit) os valores mínimo e máximo das colunas, depois transforma os dados.

In [ ]:
# Selecionar as colunas numéricas para normalizar (exceto a nova coluna price_category)
num_cols_for_scaling = ['wheel-base','length','width','height','curb-weight',
                        'engine-size','bore','stroke','compression-ratio',
                        'horsepower','peak-rpm','city-mpg','highway-mpg','price']

scaler = MinMaxScaler()
df[num_cols_for_scaling] = scaler.fit_transform(df[num_cols_for_scaling])

print("\nExemplo das variáveis normalizadas:")
display(df[num_cols_for_scaling].head())

## 8. Definição de X e y
*O vetor de rótulos (`y`) será `price_category`. Em `X`, removemos tanto `price_category` quanto `price` (pois o modelo deve prever a categoria sem ver o preço real).*

**Funções e argumentos**:
- `df.drop(columns=[...])`: remove colunas específicas do dataframe.

In [ ]:
y = df['price_category'].values
X = df.drop(columns=['price_category','price'])

print(f"\nDimensão de X: {X.shape}")
print(f"Dimensão de y: {y.shape}")

## 9. Seleção de Variáveis com RFE
*Usamos `LogisticRegression` como estimador para o RFE, a fim de selecionar as 10 melhores variáveis (características) para classificar `price_category`.*

**Funções e argumentos**:
- `LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=500)`: modelo de regressão logística multiclasse, usando o método `lbfgs` e limite de 500 iterações.
- `RFE(estimator=..., n_features_to_select=10)`: elimina recursivamente as variáveis menos importantes, até restarem 10 selecionadas.
- `rfe.fit(X, y)`: executa o processo de seleção no conjunto de dados.
- `rfe.support_`: array booleano indicando quais colunas foram mantidas.

In [ ]:
base_estimator = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=500)
rfe = RFE(estimator=base_estimator, n_features_to_select=10)

rfe.fit(X, y)

selected_columns = X.columns[rfe.support_]
print("\nColunas selecionadas pelo RFE:")
print(selected_columns)

# Criar novo X somente com as colunas selecionadas
X_selected = X[selected_columns]

## 10. Divisão em Treino e Teste
*Dividimos o dataset em 70% treino e 30% teste, estratificando pela variável alvo (`price_category`).*

**Funções e argumentos**:
- `train_test_split(X_selected, y, test_size=0.30, stratify=y, random_state=42)`: separa dados em treino e teste; 30% vai para teste. `stratify=y` mantém a proporção de classes, `random_state=42` define uma semente para reprodutibilidade.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.30, random_state=42, stratify=y
)

print("\nDimensões do conjunto de treino:", X_train.shape, y_train.shape)
print("Dimensões do conjunto de teste:", X_test.shape, y_test.shape)

## 11. Treinamento da Rede Neural (MLPClassifier)
*Definimos um `MLPClassifier` com duas camadas ocultas (16 e 8 neurônios), função de ativação `'relu'` e solver `'adam'`. Podemos ajustar o número de iterações (`max_iter`) caso seja necessário mais tempo para convergir.*

**Funções e argumentos**:
- `MLPClassifier(hidden_layer_sizes=(16,8), activation='relu', solver='adam', max_iter=2000, random_state=42)`: cria uma rede neural com 2 camadas ocultas, cada qual contendo 16 e 8 neurônios, respectivamente. A função de ativação `'relu'` (Rectified Linear Unit) é uma das mais usadas em redes neurais. O `'adam'` é um método de otimização. `max_iter=2000` define o número máximo de épocas (iterações) de treinamento. `random_state=42` garante reprodutibilidade dos resultados.
- `mlp.fit(X_train, y_train)`: treina (ajusta) os pesos da rede neural nos dados de treino.

In [ ]:
mlp = MLPClassifier(
    hidden_layer_sizes=(16, 8),
    activation='relu',
    solver='adam',
    max_iter=2000,
    random_state=42
)

mlp.fit(X_train, y_train)

## 12. Avaliação do Modelo
*Utilizamos a rede treinada para prever no conjunto de teste. Avaliamos por meio da matriz de confusão, relatório de classificação (contendo precisão, recall e F1), e acurácia geral.*

**Funções e argumentos**:
- `mlp.predict(X_test)`: gera as previsões de classe para cada amostra do conjunto de teste.
- `confusion_matrix(y_test, y_pred, labels=[...])`: produz uma matriz de confusão, onde `labels` define a ordem das classes.
- `sns.heatmap(...)`: usada para plotar a matriz de confusão com anotações.
- `classification_report(...)`: exibe precisão, recall, F1-score e suporte para cada classe.
- `accuracy_score(...)`: exibe a fração de acertos (quantos % das amostras foram classificadas corretamente).

In [ ]:
y_pred = mlp.predict(X_test)

print("\nMatriz de Confusão:")
cm = confusion_matrix(y_test, y_pred, labels=['low','medium','high'])
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['low','medium','high'],
            yticklabels=['low','medium','high'])
plt.title("Matriz de Confusão (Teste) - MLP")
plt.xlabel("Previsto")
plt.ylabel("Verdadeiro")
plt.show()

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, digits=3))

acc = accuracy_score(y_test, y_pred)
print(f"Acurácia no teste: {acc*100:.2f}%")

## 13. Conclusões
*Resumo dos resultados e possíveis pontos de melhoria.*

- **Rede Neural MLP**: demonstrou capacidade de separar bem as faixas de preço dos automóveis (baixo, médio, alto) após o pré-processamento e seleção de variáveis.
- **RFE**: mesmo utilizando uma regressão logística como estimador, ajudou a reduzir a dimensionalidade, selecionando os atributos mais relevantes.
- **Métricas**: acurácia, precisão, recall e F1-score podem indicar se o modelo está equilibrado entre as classes. Uma boa prática é analisar se há confusão excessiva entre *low* e *medium*, ou entre *medium* e *high*.
- Poderíamos **ajustar hiperparâmetros** (como número de neurônios, camadas, função de ativação, taxa de aprendizado) para melhorar ainda mais o desempenho.
- **Interpretação**: apesar de redes neurais serem modelos mais "caixa-preta", a abordagem utilizada (RFE + MLP) ainda pode fornecer insights sobre quais *features* são mais importantes (via RFE) e como afetam a classificação de preço.